In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import openpyxl as op

In [2]:
# Paramétrage :

# from final_tab.ipynb
# Define the angle range and step
start_angle = -60
end_angle = 60
step_angle = 5

# Create the array of angles
angles = np.arange(start_angle, end_angle + 1, step_angle, dtype=object)
titres_colonnes = np.append(angles, np.array("temoin", dtype=object), axis=None)

# Annual parameters
###### # from final_tab.ipynb
start_date = datetime(year = 2023, month = 1, day = 1, hour = 0)
end_date = datetime(year = 2024, month = 1, day = 1, hour = 0)
print((end_date-start_date).days)

# Hourly parameters
###### # from final_tab.ipynb
HEURE_DEB = 6
HEURE_FIN = 22
HEURE_BLANCHES = 24 - (HEURE_FIN-HEURE_DEB)
NB_HEURES_ANNEES_ETUD = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days
TOT_HOYS = (end_date-start_date).days*24

365


In [3]:
# Read the generated table in previous steps
df = pd.read_excel("results/6-22H_step5_semitransp.xlsx",sheet_name="Sheet1",header=None)
df.columns = titres_colonnes
print(df)

           -60       -55       -50       -45       -40       -35       -30  \
0     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1     0.005105  0.004488  0.006516  0.005690  0.005727  0.003359  0.003568   
2     0.020128  0.017694  0.019556  0.021985  0.022132  0.018927  0.019753   
3     0.124798  0.120060  0.046643  0.057511  0.130782  0.128601  0.117584   
4     0.292718  0.267977  0.212828  0.215884  0.289251  0.286339  0.174272   
...        ...       ...       ...       ...       ...       ...       ...   
5835  0.000145  0.000128  0.000126  0.000102  0.000104  0.000095  0.000101   
5836  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5837  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5838  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5839  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

           -25       -20       -15  ...        20        25    

In [5]:
# Step 1: Add as many null lines as HEURE_DEB if positive
if HEURE_DEB > 0:
    null_rows_start = pd.DataFrame(0, index=range(HEURE_DEB), columns=titres_colonnes)
    df2 = pd.concat([null_rows_start, df], ignore_index=True)

# Step 2: Add 8 empty lines every 16 lines until 8760 lines
if HEURE_BLANCHES > 0:
    empty_rows = pd.DataFrame(0, index=range(HEURE_BLANCHES), columns=titres_colonnes)

    for idx in range(HEURE_FIN, TOT_HOYS, 24):
        df2 = pd.concat([df2.iloc[:idx], empty_rows, df2.iloc[idx:]], ignore_index=True)

# Step 3: Remove excess lines if any
if df2.shape[0] > TOT_HOYS :
    surplus = df2.shape[0] - TOT_HOYS
    df3 = df2.iloc[:-surplus]

In [8]:
print(df3[24:45])

         -60       -55       -50       -45       -40       -35       -30  \
24  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
25  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
26  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
27  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
28  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
29  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
30  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
31  0.001646  0.001447  0.001459  0.001192  0.001204  0.001083  0.001151   
32  0.005176  0.004550  0.004523  0.003712  0.003750  0.003433  0.003645   
33  0.008426  0.007434  0.007155  0.005885  0.006133  0.005676  0.005981   
34  0.008829  0.007789  0.007550  0.006241  0.006490  0.006030  0.006115   
35  0.006246  0.005492  0.005466  0.004510  0.004554  0.004225  0.004244   
36  0.004770

In [10]:
# Create a DataFrame with the hours of the year (from 0 to 8759) as an index
hours_in_year = range(8760)
dfX = pd.DataFrame(index=hours_in_year)

# Add the 'month_day_hour' column
df3['month_day_hour'] = pd.to_datetime(dfX.index.map(lambda x: start_date + timedelta(hours=x)))

# Rearrange columns in df3
cols = ['month_day_hour'] + [col for col in df3 if col != 'month_day_hour']
df3 = df3.loc[:, cols]
print(df3.head())

       month_day_hour  -60  -55  -50  -45  -40  -35  -30  -25  -20  ...   20  \
0 2023-01-01 00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1 2023-01-01 01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2 2023-01-01 02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3 2023-01-01 03:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4 2023-01-01 04:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

    25   30   35   40   45   50   55   60  temoin  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  

[5 rows x 27 columns]


Import scenario :
1 = True = Backtracking (productible max)
0 = False = Effacement (irradiance au sol maximum)

In [23]:
from scenario import Scenario

## TESTS ##
# Instancier des objets scénarios
s1 = None
xl_path = "scenarios/sc_test1.xlsx"
s1 = Scenario(file_path=xl_path)
# print(s1.scenario[:,:])

s2 = Scenario(file_path=xl_path)
print(s2.scenario[:,:])



[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False  True  True  True False 

In [24]:
s2.scenario.shape

(12, 24)

Création équivalent 1/0

In [25]:
# Création_tab 1
sc10 = np.full((12,24), 0, dtype = "int8")

for i in range(s2.scenario.shape[0]):  # Pour chaque ligne
    for j in range(s2.scenario.shape[1]):  # Pour chaque colonne
        if s2.scenario[i, j] == True:
            sc10[i, j] = 1

print(sc10)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Création tab_1

In [26]:
# Nombre de lignes souhaité
n_rows = 8760

# Création d'un DataFrame avec des valeurs par défaut (par exemple, 0)
tab_1 = pd.DataFrame({
    'hoys': df3['month_day_hour'],
    'effacement': np.ones(n_rows),
    'angle': np.zeros(n_rows),
    'irradiance_sol': np.zeros(n_rows)
})


Import PVSyst

In [27]:
# Chemin vers le fichier Excel
file_path = 'PVSystTestFile.xlsm'

# Nom de l'onglet à lire
sheet_names = ['Tests','Illim']

# Lire l'onglet spécifique
pvsyst = pd.read_excel(file_path, sheet_name=sheet_names)

In [28]:
pvsyst_illim = pvsyst['Illim'].drop(index=list(range(12))).drop(columns=['Unnamed: 4', 'Unnamed: 5']).reset_index(drop=True)
pvsyst_illim.columns = ['Date','E_Grid','PR','PhiAng']
print(pvsyst_illim)

                     Date E_Grid PR PhiAng
0     1990-01-01 00:00:00 -3.048  0      0
1     1990-01-01 01:00:00 -3.048  0      0
2     1990-01-01 02:00:00 -3.048  0      0
3     1990-01-01 03:00:00 -3.048  0      0
4     1990-01-01 04:00:00 -3.048  0      0
...                   ...    ... ..    ...
8755  1990-12-31 19:00:00 -3.048  0      0
8756  1990-12-31 20:00:00 -3.048  0      0
8757  1990-12-31 21:00:00 -3.048  0      0
8758  1990-12-31 22:00:00 -3.048  0      0
8759  1990-12-31 23:00:00 -3.048  0      0

[8760 rows x 4 columns]


In [29]:
pvsyst_tests = pvsyst['Tests'].drop(index=list(range(12))).drop(columns=pvsyst['Tests'].columns[4:]).reset_index(drop=True)
pvsyst_tests.columns = ['Date', 'PhiAng', 'E_Grid_SunTrack', 'Egrid Effectif']
print(pvsyst_tests)

                     Date PhiAng E_Grid_SunTrack Egrid Effectif
0     1990-01-01 00:00:00      0          -3.048              1
1     1990-01-01 01:00:00      0          -3.048              1
2     1990-01-01 02:00:00      0          -3.048              1
3     1990-01-01 03:00:00      0          -3.048              1
4     1990-01-01 04:00:00      0          -3.048              1
...                   ...    ...             ...            ...
8755  1990-12-31 19:00:00      0          -3.048             12
8756  1990-12-31 20:00:00      0          -3.048             12
8757  1990-12-31 21:00:00      0          -3.048             12
8758  1990-12-31 22:00:00      0          -3.048             12
8759  1990-12-31 23:00:00      0          -3.048             12

[8760 rows x 4 columns]


Remplir le tab_1 :

In [30]:
# Remplissage de la colonne 'effacement'
# Parcourir scénario, pour chaque case 0 (False), changer les 1 (panneaux en suntracking) de la colonne 'effacement' en 0 (panneaux effacés)

for i in range(s1.scenario.shape[0]) :  # Pour chaque ligne
    for j in range(s1.scenario.shape[1]) :  # Pour chaque colonne
        if s1.scenario[i,j] == False :
            tab_1.loc[(tab_1['hoys'].dt.hour == j) & (tab_1['hoys'].dt.month == i+1), 'effacement'] = 0

In [31]:
# Remplissage de la colonne 'angle'
# Parcourir tableau_1, en fonction de 'effacement', remplir 'angle' et 'irradiance'
for i in range(tab_1.shape[0]) :
    if tab_1.at[i,'effacement'] == 0:
        #aller chercher irradiance au sol maximum dans tab_final (df3 dans ce .ipynb), et retrouver l'angle correspondant (nom de colonne)
        tab_1.loc[i,'irradiance_sol'] = df3.drop(columns='month_day_hour').iloc[i].max() # Irradiance max
        tab_1.loc[i,'angles'] = df3.drop(columns='month_day_hour').iloc[i].idxmax() # Colonne dont l'irradiance est max
        # /!\ Si égalité entre plusieurs angles, va choisir l'angle le moins élevé (-60 à priori)
    else: #'effacement' == 1
        # pvsyst_illim.loc[...] = angle PVSYST optimal, arrondir, puis trouver irradiance au sol correspondante
        tab_1.loc[i,'irradiance_sol'] = df3.at[i,round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle]
        tab_1.loc[i,'angles'] = round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle

# Tableau_1 rempli, il ne reste qu'à calculer les rapports entre zone témoin /!\(à intégrer)/!\ et zone d'étude
  

KeyError: 0

In [ ]:
# print(tab_1[(tab_1['effacement'] < 1) & (tab_1['hoys'].dt.month == 9)])
print(tab_1)

                    hoys  effacement  angle  irradiance_sol  angles
0    2023-01-01 00:00:00         1.0    0.0             0.0     0.0
1    2023-01-01 01:00:00         1.0    0.0             0.0     0.0
2    2023-01-01 02:00:00         1.0    0.0             0.0     0.0
3    2023-01-01 03:00:00         1.0    0.0             0.0     0.0
4    2023-01-01 04:00:00         1.0    0.0             0.0     0.0
...                  ...         ...    ...             ...     ...
8755 2023-12-31 19:00:00         1.0    0.0             0.0     0.0
8756 2023-12-31 20:00:00         1.0    0.0             0.0     0.0
8757 2023-12-31 21:00:00         1.0    0.0             0.0     0.0
8758 2023-12-31 22:00:00         1.0    0.0             0.0     0.0
8759 2023-12-31 23:00:00         1.0    0.0             0.0     0.0

[8760 rows x 5 columns]


Calcul liste de taille 365 concernant rapport de irradiance_zone_témoin sur irradiance_zone_étude

In [ ]:
df_daily = tab_1.groupby(tab_1['hoys'].dt.date)['irradiance_sol'].sum().reset_index()
df_daily.columns = ['date', 'irradiance_sol_day']

print(df_daily)

           date  irradiance_sol_day
0    2023-01-01            2.444301
1    2023-01-02            0.339211
2    2023-01-03            3.506907
3    2023-01-04            2.824320
4    2023-01-05            0.552571
..          ...                 ...
360  2023-12-27            1.630395
361  2023-12-28            0.160914
362  2023-12-29            1.213723
363  2023-12-30            3.113947
364  2023-12-31            3.771072

[365 rows x 2 columns]


###########
###########
#### TESTS
###########
###########

In [33]:
df3[5:25]

,month_day_hour,-60,1,temoin
5,2023-01-01 05:00:00,0.000000,0.000000,0.000000
6,2023-01-01 06:00:00,0.000000,0.000000,0.000000
7,2023-01-01 07:00:00,0.049168,0.049168,0.049168
8,2023-01-01 08:00:00,0.181327,0.181327,0.181327
9,2023-01-01 09:00:00,0.312087,0.312087,0.312087
10,2023-01-01 10:00:00,0.343043,0.343043,0.343043
11,2023-01-01 11:00:00,0.305425,0.305425,0.305425
12,2023-01-01 12:00:00,0.320081,0.320081,0.320081
13,2023-01-01 13:00:00,0.376153,0.376153,0.376153
14,2023-01-01 14:00:00,0.334291,0.334291,0.334291
